In [1]:
import os 
import sys

spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.4-src.zip'))
sys.path.insert(0, os.path.join(spark_home, 'python'))
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.0
      /_/

Using Python version 2.7.11 (default, Dec  6 2015 18:57:58)
SparkSession available as 'spark'.


In [2]:
DatasetRaw = sc.textFile('114_congress.csv')

def remove_header(itr_index, itr):
    return iter(list(itr)[1:]) if itr_index == 0 else itr

DatasetRaw = DatasetRaw.mapPartitionsWithIndex(remove_header).cache()

In [3]:
DatasetRaw = DatasetRaw.map(lambda line: [str(i + 1) + ":" + x + " " for i, x in enumerate(line.split(',')[3:])])

In [5]:
DatasetRaw = DatasetRaw.zipWithIndex().map(lambda line: [str(line[1]) + ' '] + line[0]).map(lambda line: (1,line + ['\n'])).map(lambda line: (line[0],''.join(line[1])))

In [ ]:
NewDataset = DatasetRaw.reduceByKey(lambda v1, v2: v1+v2).collect()

In [ ]:
encoding = 'utf-8'
import io

with io.open('new_votes.txt', 'w', encoding=encoding) as file:
    file.write(NewDataset)

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import PCA, StandardScaler

#Kmeans
dataset = spark.read.format("libsvm").load("new_votes.txt")
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(dataset)

In [ ]:
#PCA
standardizer = StandardScaler(withMean=True, withStd=True, inputCol='features', outputCol='features_2')
model = standardizer.fit(dataset)
dataset = model.transform(dataset)
pca = PCA(k=2, inputCol='features_2', outputCol='pcaFeatures')
model = pca.fit(dataset)
result = model.transform(dataset).select('pcaFeatures')
result.show(truncate=False)

# Putting all together
predictions = model.transform(dataset)